In [1]:
import numpy as np
from numpy import allclose as eq, conj, array, complex128

V2 = np.sqrt(2)

# Basis

In [2]:
basis_0 = array([[1, 0]], dtype=complex).T
basis_1 = array([[0, 1]], dtype=complex).T

In [3]:
plus = (basis_0 + basis_1) / V2
minus = (basis_0 - basis_1) / V2

In [4]:
R = (basis_0 + 1j * basis_1) / V2
L = (basis_0 - 1j * basis_1) / V2

In [5]:
assert eq(conj(basis_0).T @ basis_1, 0)
assert eq(conj(plus).T @ minus, 0)
assert eq(conj(R).T @ L, 0)

# Hadamard Gate

In [6]:
H = array([[1, 1],
           [1, -1]], dtype=complex128) / V2

In [7]:
assert eq(conj(H) @ H, np.eye(2))
assert eq(H @ basis_0, plus)
assert eq(H @ plus, basis_0)

# Pauli Matrices

In [8]:
pauli_X = array([[0, 1],
                 [1, 0]], dtype=complex)

pauli_Y = array([[0, -1j],
                 [1j, 0]], dtype=complex)

pauli_Z = array([[1, 0],
                 [0, -1]], dtype=complex)

# Measurements

In [9]:
w, v = np.linalg.eig(pauli_Z)
v_0, v_1 = np.expand_dims(v, axis=-1)

for i in range(w.shape[0]):
    print (f"{i}: {str(np.round(w[i],10)):7s} -> {v[:,i]}")

assert eq(abs(v_0.conj().T @ basis_0), 1)
assert eq(abs(v_1.conj().T @ basis_1), 1)

0: (1+0j)  -> [1.+0.j 0.+0.j]
1: (-1+0j) -> [0.+0.j 1.+0.j]


In [10]:
w, v = np.linalg.eig(pauli_X)
v_0, v_1 = np.expand_dims(v, axis=-1)

for i in range(w.shape[0]):
    print (f"{i}: {str(np.round(w[i],10)):7s} -> {v[:,i]}")

assert eq(abs(v_0.conj().T @ plus), 1)
assert eq(abs(v_1.conj().T @ minus), 1)

0: (1+0j)  -> [0.70710678-0.j 0.70710678+0.j]
1: (-1+0j) -> [ 0.70710678+0.j -0.70710678-0.j]


In [11]:
w, v = np.linalg.eig(pauli_Y)
v_0, v_1 = np.expand_dims(v, axis=-1)

for i in range(w.shape[0]):
    print (f"{i}: {str(np.round(w[i],10)):7s} -> {v[:,i]}")

assert eq(abs(v_0.conj().T @ R), 1)
assert eq(abs(v_1.conj().T @ L), 1)

0: (1+0j)  -> [-0.        -0.70710678j  0.70710678+0.j        ]
1: (-1+0j) -> [0.70710678+0.j         0.        -0.70710678j]


## Expectation value

In [12]:
assert eq(basis_0.conj().T @ pauli_Z @ basis_0, 1)   # ⟨0|Z|0⟩ = 1
assert eq(basis_1.conj().T @ pauli_Z @ basis_1, -1)  # ⟨1|Z|1⟩ = -1
assert eq(plus.conj().T @ pauli_Z @ plus, 0)  # ⟨+|Z|+⟩ = 0
assert eq(plus.conj().T @ pauli_X @ plus, 1)  # ⟨+|X|+⟩ = 1
assert eq(R.conj().T @ pauli_Y @ R, 1)   # ⟨R|Y|R⟩ = 1

## Projection operator

In [13]:
w, v = np.linalg.eig(pauli_Z)
v_0, v_1 = np.expand_dims(v, axis=-1)

proj_0 = v_0 @ v_0.T   
proj_1 = v_1 @ v_1.T     

In [14]:
q = H @ basis_0

p_0 = q.conj().T @ proj_0 @ q
p_1 = q.conj().T @ proj_1 @ q

print (p_0, p_1)

[[0.5+0.j]] [[0.5+0.j]]


# Gates
## T Gate

In [15]:
T_gate = array([[1, 0],
                [0, (1+1j)/V2]], dtype=complex)

assert eq(T_gate.conj().T @ T_gate, np.eye(2))

## RY Gate

In [134]:
def ry(phi):
    return array([
        [np.cos(phi / 2), -np.sin(phi / 2)],
        [np.sin(phi / 2),  np.cos(phi / 2)]
    ], dtype=complex)

assert eq(ry(np.pi) @ basis_0, basis_1)
assert eq(ry(-np.pi) @ basis_1, basis_0)
assert eq(ry(-np.pi/2) @ basis_0, minus)
assert eq(ry(np.pi/2) @ basis_0, plus)


In [68]:
random_state = np.array([0.42 + 0j, np.sqrt(1 - 0.42**2) + 0j]).reshape((2, 1))

In [88]:
def measurement(op):
    return lambda state: state.conj().T @ op @ state

In [114]:
state  = array([[0.42 + 0j], [np.sqrt(1 - 0.42**2) + 0j]]) # random state
assert eq(abs(state.conj().T @ state), 1)

X = measurement(pauli_X)
Y = measurement(pauli_Y)
Z = measurement(pauli_Z)

assert eq(X(state), X(ry(np.pi*2) @ state))
assert eq(Z(state), Z(ry(np.pi*2) @ state))

assert eq(X(plus), X(ry(np.pi/2) @ basis_0))
assert eq(Z(plus), Z(ry(np.pi/2) @ basis_0))

assert eq(X(minus), X(ry(-np.pi/2) @ basis_0))
assert eq(Z(minus), Z(ry(-np.pi/2) @ basis_0))

## RZ Gate

In [135]:
def rz(phi):
    return array([
        [np.exp(-1j * phi / 2), 0],
        [0,  np.exp(1j * phi / 2)]
    ], dtype=complex)

assert eq(X(rz(np.pi / 2) @ plus), X(R))
assert eq(Y(rz(np.pi / 2) @ plus), Y(R))
assert eq(Z(rz(np.pi / 2) @ plus), Z(R))


## RX Gate

In [170]:
def rx(phi):
    return array([
        [np.cos(phi / 2), -1j * np.sin(phi / 2)],
        [-1j * np.sin(phi / 2),  np.cos(phi / 2)]
    ], dtype=complex)

assert eq(X(rx(-np.pi/2) @ basis_0), X(R))
assert eq(Y(rx(-np.pi/2) @ basis_0), Y(R))
assert eq(Z(rx(-np.pi/2) @ basis_0), Z(R))

assert eq(X(rx(np.pi/2) @ basis_0), X(L))
assert eq(Y(rx(np.pi/2) @ basis_0), Y(L))
assert eq(Z(rx(np.pi/2) @ basis_0), Z(L))


In [171]:
eq(rx(np.pi) @ ry(np.pi/2), H)

False

In [173]:
ry(np.pi/2)

array([[ 0.70710678+0.j, -0.70710678+0.j],
       [ 0.70710678+0.j,  0.70710678+0.j]])